# Train a regressor on a set of embeddings of tweet texts

Use **GetOldTweets3** library (available via Pypi)

In [1]:
!pip install GetOldTweets3
import GetOldTweets3 as got

In [2]:
# !pip install basilica # might have to install, if not available in underlying environment

In [28]:
import pandas as pd
import json
import numpy as np
import sklearn.linear_model
import sklearn.preprocessing
import sklearn.decomposition
import basilica

In [5]:
twitter_user_name = 'LambdaSchool'
count = 1000 # during testing

In [6]:
#  Create object to execute queries
querySpecs = got.manager.TweetCriteria().setUsername(twitter_user_name).setMaxTweets(count)

In [7]:
# get a set of tweets
retrieved_tweets = got.manager.TweetManager.getTweets(querySpecs)
len(retrieved_tweets)

1000

In [29]:
# df.head()

In [17]:
# we only need the texts for making embeddings
test_texts = [tweet.text for tweet in retrieved_tweets]
len(test_texts)

1000

In [18]:
with basilica.Connection('SLOW_DEMO_KEY') as c:
    embeddings = list(c.embed_sentences(test_texts))
len(embeddings)

1000

In [19]:
# Normalize to help regressor work better
normalized_embeddings = sklearn.preprocessing.normalize(embeddings)
len(normalized_embeddings)
# normalized_embeddings[0]

# put the normalized embeddings back in a dataframe
normalized_embeddings_df = pd.DataFrame(normalized_embeddings)
# PCA the embeddings
# sklearn.get_config()
normalized_embeddings_df.shape

(1000, 768)

## Split into X matrix (embeddings) and y vector (retweet count)

In [33]:
X = normalized_embeddings_df
y =df['retweets']
print(y.shape)
print(y.isna().value_counts())

(1000,)
False    1000
Name: retweets, dtype: int64


In [21]:
X.shape

(1000, 768)

## Training a regressor

In [22]:
# import numpy as np
# import sklearn.linear_model
# import sklearn.preprocessing
# import sklearn.model_selection

In [23]:
X_train, X_test = sklearn.model_selection.train_test_split(X, random_state=72)
y_train, y_test = sklearn.model_selection.train_test_split(y, random_state=72)

model = sklearn.linear_model.LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

## Results

In [24]:
print('Train accuracy: %.3f' % model.score(X_train, y_train))
print('Test accuracy: %.3f' % model.score(X_test, y_test))

Train accuracy: 0.327
Test accuracy: 0.320


In [25]:
# y[0:11]

id
1250135493662171136    1
1250084165405597702    3
1249834817878986753    3
1249797109382967296    2
1249456259503403013    0
1249389342663065600    0
1249029951602425857    1
1248781591553298432    2
1248719916955549701    5
1248606813194522624    2
1248362603321143297    6
Name: retweets, dtype: int64